# Advertising Campaign Report - Sept/Oct 2017

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pdfkit

# Cleaning

In [2]:
purchase_data = pd.read_excel("./dataset.xlsx", sheet_name='Purchases')
airings_data = pd.read_excel("./dataset.xlsx", sheet_name='Airings')

# The first row of Lookup table says "Lookup table for survey response field to airings network ticker symbol."  Assuming the first row always says that, we can drop it.
lookup_data = pd.read_excel("./dataset.xlsx", sheet_name='Lookup', skiprows=1)

# Preprocessing

In [3]:
# Lookup data is meant to facilitate a join between the purchases data and the spend/lift data, but lookup data has a row with all null values, which doesn't help the join in any way.  I'll drop that row.
lookup_data = lookup_data.dropna(how='all')

# The Network Name column and Network Name.1 column contain identical information, so we can drop the duplicated column.
lookup_data.drop(labels='Network Name.1', axis=1, inplace=True)

In [4]:
# In order to make sure the joins happen correctly, we need to ensure that the strings we're joining on actually match.

# The purchase data table is pretty messy, but if we assume the second column always contains the names of the networks, we can use .iloc to grab them and ensure they're lowercase 

lookup_data['Network Name'] = lookup_data['Network Name'].str.lower()
lookup_data['Ticker'] = lookup_data['Ticker'].str.upper()
airings_data['Network'] = airings_data['Network'].str.upper()
purchase_data.iloc[:, 1] = purchase_data.iloc[:, 1].str.lower()

# Transposing Purchase Exit Survey Data - Converting dates from columns to rows in Purchase Exit Survey Data

The purchases table is extremely messy and would be easier to work with if the rows were dates and the columns were the networks.  If we assume that the first row will always have the year, the third row will always have month names, and the fourth row will always have the day numbers, I can programmatically concatenate all the necessary date information in the form "Year-Month-Day", then use them for the rows.

In [5]:
current_year = purchase_data.iloc[0,:].dropna()
current_year = int(current_year)
current_year

2017

In [6]:
months = []
for month in purchase_data.iloc[2,2:].dropna():
    months.append(month)
months

['September', 'October']

In [7]:
# Grab the row of day numbers and cast as integers
day_nums = np.array(purchase_data.iloc[3,2:], dtype=int)

parsed_dates = []
current_month = months[0]
i = 0

# Walk through the list of day_nums.  

# If current_day_num > next_day_num, that indicates a change in month (ex: If current_day = Sept-30 and next_day = Oct-1, b/c 30 > 1).  When this happens, we concatenate the current_day, then increase i by 1 to set the current_month to the next month for furture concatenation.  

# If current_day_num < next_day_num, that indicates both days are in the same month (ex: If current_day = Sept-5 and next_day = Sept-6, b/c 5 < 6), so we concatenate like normal.

# The try block handles the exception when you get to the last day in day_nums.  Since there are no more days in the list, we get an error when we try to index into the list one day into the future.
for count, current_day_num in enumerate(day_nums, start=1):
    try:
        next_day_num = day_nums[count]
    except:
        pass
    if current_day_num > next_day_num:
        current_date = str(current_year) + '-' + current_month + '-' + str(current_day_num)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        i += 1
        current_month = months[i]
        parsed_dates.append(current_date)
    else:
        current_date = str(current_year) + '-' + current_month + '-' + str(current_day_num)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        parsed_dates.append(current_date)

# Now that all the dates have been parsed, we replace the unparsed dates with the parsed ones, then transpose the table.  We now have rows that correspond to dates and columns that correspond to networks
purchase_data.iloc[3,2:] = parsed_dates
purchase_data_transpose = purchase_data.iloc[3:,:].transpose()

## Some Cleanup

In [8]:
# Set the column of dates as the index and rename the axis appropriately
purchase_data_transpose.set_index(3, inplace=True)
purchase_data_transpose.rename_axis('date', inplace=True)

# Drop first row, which doesn't contain anything useful
purchase_data_transpose = purchase_data_transpose.iloc[1:]

# Replace column names with the row of network names and then drop that row
purchase_data_transpose.columns = purchase_data_transpose.iloc[0]
purchase_data_transpose = purchase_data_transpose.drop(labels='source')

# Rename column axis as upper-case "Source" to match original table
purchase_data_transpose.rename_axis('Source', axis='columns', inplace=True)

# Convert index of dates to datetime objects
purchase_data_transpose.index = pd.to_datetime(purchase_data_transpose.index)

## Done

# Metrics by Network

## Purchases by Network

In [9]:
# Aggregate across rows to find the total number of purchases by network
purchases_by_network = purchase_data_transpose.sum(axis=0)
purchases_by_network = purchases_by_network.to_frame()
purchases_by_network = purchases_by_network.rename(columns={0:'Purchases'})

## Spend and Lift by Network

In [10]:
# Aggregate to find the total amount spent on ads for each network and the total lift generated from those ads for each network 
spend_and_lift_by_network = airings_data.groupby('Network')[['Spend', 'Lift']].agg('sum')

## Joins

### Joining Purchases by Network to Lookup Data

In [11]:
purchases_by_network_w_lookup = lookup_data.merge(right=purchases_by_network, left_on='Network Name', right_on='Source', how='left')
purchases_by_network_w_lookup.set_index('Network Name', inplace=True)

### Joining Purchases/Lookup by Network to Spend and Lift

In [12]:
purchases_spend_lift_by_network = purchases_by_network_w_lookup.merge(right=spend_and_lift_by_network, left_on='Ticker', right_index=True, how='left')

# Since this column was only needed for the join, I'm going to drop it post join
purchases_spend_lift_by_network.drop('Ticker', axis=1, inplace=True)

In [13]:
purchases_spend_lift_by_network.index = purchases_spend_lift_by_network.index.str.replace('_', ' ').str.title()

In [14]:
purchases_spend_lift_by_network.fillna(0, inplace=True)

## Computing Metrics by Network

In [15]:
purchases_spend_lift_by_network['Conversion Rate (Purchases/Lift)%'] = purchases_spend_lift_by_network['Purchases'] / purchases_spend_lift_by_network['Lift'] * 100

purchases_spend_lift_by_network['Cost Per Acquisition (Spend/Purchases)'] = purchases_spend_lift_by_network['Spend'] / purchases_spend_lift_by_network['Purchases'].fillna(0)

purchases_spend_lift_by_network['Cost Per Visitor (Spend/Lift)'] = purchases_spend_lift_by_network['Spend'] / purchases_spend_lift_by_network['Lift'].fillna(0)

purchases_spend_lift_by_network['Percent of Purchases'] = purchases_spend_lift_by_network['Purchases'] / sum(purchases_spend_lift_by_network['Purchases'].fillna(0)) * 100

purchases_spend_lift_by_network['Percent of Spend'] = purchases_spend_lift_by_network['Spend'] / sum(purchases_spend_lift_by_network['Spend'].fillna(0)) * 100

purchases_spend_lift_by_network['Percent Pur > Percent Spend'] = purchases_spend_lift_by_network['Percent of Purchases'] > purchases_spend_lift_by_network['Percent of Spend']

## Output results to CSV file

In [16]:
current_year_and_months = str(current_year) + '_' + '_'.join(str(month) for month in months)

purchases_spend_lift_by_network.to_csv(F"./output/cleaned_csvs/purchases_spend_lift_by_network_{current_year_and_months}.csv")

## Done

# Metrics by Network and Month

## Creating a lookup table that has monthly date information through a Cross Join

In [17]:
# First we generate a series that has the monthly date information as the index, which we can grab
month_stamps = purchase_data_transpose.groupby(pd.Grouper(freq='M')).sum().index.values

# Then we convert the series to a DataFrame and add a key column, which will be used in the cross join.  Pandas doesn't have built in cross join functionality, so this will be used as a work around
month_df = pd.DataFrame(data=month_stamps)
month_df['key']=0

# If we also add the same key to our lookup data, we'll be able to join the months in our spreadsheet to each row of the lookup table, and merge them together on that key, effectively creating a cross join
lookup_data_with_key = lookup_data.copy()
lookup_data_with_key['key'] = 0


lookup_data_with_months = lookup_data_with_key.merge(month_df)
lookup_data_with_months.rename(columns={0:'date'}, inplace=True)
lookup_data_with_months.drop(columns='key', inplace=True)

# Now we have a lookup table that has the appropriate dates for each month in the campaign for each channel

## Aggregating Spend and Lift by Network and Month

In [18]:
spend_lift_by_network_and_month = airings_data.groupby(['Network', pd.Grouper(key='Date/Time ET', freq='M')]).sum().reset_index()

## Aggregating Purchases by Network and Month

In [19]:
purchases_by_network_and_month = purchase_data_transpose.groupby(pd.Grouper(freq='M')).sum().transpose().stack().to_frame().reset_index()

purchases_by_network_and_month.rename(columns={0:'Purchases'}, inplace=True)

## Joins

### Joining lookup_data_with_months to spend_lift_by_network_and_month

In [20]:
lookup_data_with_months.head()

,Network Name,Ticker,date
0,(blank),NaN,2017-09-30
1,(blank),NaN,2017-10-31
2,aapka_colors,NaN,2017-09-30
3,aapka_colors,NaN,2017-10-31
4,baby_first,BABY,2017-09-30


In [21]:
spend_lift_by_network_and_month.head()

,Network,Date/Time ET,Creative,Spend,Lift
0,BLOM,2017-09-30,53535353,4966.72,199
1,CNBC,2017-09-30,14141414,6481.42,406
2,CNBCWORLD,2017-10-31,114141413,1299.99,51
3,CNN,2017-09-30,7070707,9159.60,507
4,CNN,2017-10-31,6060606,8954.75,574


In [22]:
lookup_spend_lift_by_network_and_month = lookup_data_with_months.merge(spend_lift_by_network_and_month, left_on=['Ticker', 'date'], right_on=['Network', 'Date/Time ET'], how='left')

lookup_spend_lift_by_network_and_month.drop(columns=['Ticker', 'Network', 'Date/Time ET'], inplace=True)

### Joining Spend and Lift to Purchases

In [23]:
purchases_spend_lift_by_network_and_month = lookup_spend_lift_by_network_and_month.merge(purchases_by_network_and_month, left_on=['Network Name', 'date'], right_on=['Source', 'date'], how='left')

purchases_spend_lift_by_network_and_month.drop(columns='Source', inplace=True)

## Cleanup

In [24]:
purchases_spend_lift_by_network_and_month['Network Name'] = purchases_spend_lift_by_network_and_month['Network Name'].str.replace('_', ' ').str.title()

In [25]:
purchases_spend_lift_by_network_and_month.head()

,Network Name,date,Creative,Spend,Lift,Purchases
0,(Blank),2017-09-30,NaN,NaN,NaN,0.0
1,(Blank),2017-10-31,NaN,NaN,NaN,3.0
2,Aapka Colors,2017-09-30,NaN,NaN,NaN,1.0
3,Aapka Colors,2017-10-31,NaN,NaN,NaN,5.0
4,Baby First,2017-09-30,NaN,NaN,NaN,1.0


In [26]:
purchases_spend_lift_by_network_and_month = purchases_spend_lift_by_network_and_month.set_index(['Network Name', 'date'])

In [27]:
purchases_spend_lift_by_network_and_month.fillna(0, inplace=True)

In [28]:
purchases_spend_lift_by_network_and_month = purchases_spend_lift_by_network_and_month[['Purchases', 'Spend', 'Lift']]

## Computing Metrics by Network and Month

In [29]:
purchases_spend_lift_by_network_and_month['Conversion Rate (Purchases/Lift)%'] = purchases_spend_lift_by_network_and_month['Purchases'] / purchases_spend_lift_by_network_and_month['Lift'] * 100

purchases_spend_lift_by_network_and_month['Cost Per Acquisition (Spend/Purchases)'] = purchases_spend_lift_by_network_and_month['Spend'] / purchases_spend_lift_by_network_and_month['Purchases']

purchases_spend_lift_by_network_and_month['Cost Per Visitor (Spend/Lift)'] = purchases_spend_lift_by_network_and_month['Spend'] / purchases_spend_lift_by_network_and_month['Lift']

purchases_spend_lift_by_network_and_month['Percent of Purchases'] = purchases_spend_lift_by_network_and_month['Purchases'] / sum(purchases_spend_lift_by_network_and_month['Purchases'].fillna(0)) * 100

purchases_spend_lift_by_network_and_month['Percent of Spend'] = purchases_spend_lift_by_network_and_month['Spend'] / sum(purchases_spend_lift_by_network_and_month['Spend'].fillna(0)) * 100

purchases_spend_lift_by_network_and_month['Percent Pur > Percent Spend'] = purchases_spend_lift_by_network_and_month['Percent of Purchases'] > purchases_spend_lift_by_network_and_month['Percent of Spend']

In [30]:
purchases_spend_lift_by_network_and_month = purchases_spend_lift_by_network_and_month.round({"Purchases":0, "Spend":2, "Lift":0, "Conversion Rate (Purchases/Lift)%":1, "Cost Per Acquisition (Spend/Purchases)":2, "Cost Per Visitor (Spend/Lift)":2})

purchases_spend_lift_by_network_and_month[['Purchases', 'Lift']] = purchases_spend_lift_by_network_and_month[['Purchases', 'Lift']].astype(int)

purchases_spend_lift_by_network_and_month = purchases_spend_lift_by_network_and_month.sort_values('Network Name')

## Output results to CSV file

In [31]:
purchases_spend_lift_by_network_and_month.to_csv(F"./output/cleaned_csvs/purchases_spend_lift_by_network_and_month_{current_year_and_months}.csv")

## Done

# Generating Reports

## Overall report by network

In [32]:
report_for_client = purchases_spend_lift_by_network.drop(['Percent of Purchases', 'Percent of Spend', 'Percent Pur > Percent Spend'], axis=1)

report_for_client.query('Spend > 0', inplace=True)

In [33]:
report_for_client[['Purchases', 'Lift']] = report_for_client[['Purchases', 'Lift']].astype(int)

report_for_client = report_for_client.round({"Purchases":0, "Spend":2, "Lift":0, "Conversion Rate (Purchases/Lift)%":1, "Cost Per Acquisition (Spend/Purchases)":2, "Cost Per Visitor (Spend/Lift)":2})


report_for_client.rename_axis('Network', axis=0, inplace=True)

report_for_client = report_for_client.sort_values('Network')

## Monthly report by network

In [34]:
report_for_client_by_month = purchases_spend_lift_by_network_and_month.drop(['Percent of Purchases', 'Percent of Spend', 'Percent Pur > Percent Spend'], axis=1)

report_for_client_by_month.index.set_names('Network', level=0, inplace=True)

In [35]:
report_for_client_by_month = report_for_client_by_month.round({"Purchases":0, "Spend":2, "Lift":0, "Conversion Rate (Purchases/Lift)%":1, "Cost Per Acquisition (Spend/Purchases)":2, "Cost Per Visitor (Spend/Lift)":2})

report_for_client_by_month[['Purchases', 'Lift']] = report_for_client_by_month[['Purchases', 'Lift']].astype(int)

In [36]:
# This will ensure that both reports have the same channels.  Since we already filtered report_for_client to show only channels where there was spend, report_for_client_by_month will also also have those same channels.  
report_for_client_by_month = report_for_client_by_month.loc[report_for_client.index]

In [37]:
report_for_client_by_month.fillna(0, inplace=True)

## Report of channels where Spend = 0

In [38]:
channels_no_spend = purchases_spend_lift_by_network.query('Spend == 0')['Purchases'].to_frame()

channels_no_spend.rename_axis('Network', axis=0, inplace=True)

channels_no_spend.sort_values(by='Purchases', ascending=False, inplace=True)

## Output reports to CSV file

In [39]:
report_for_client.to_csv(F"./output/cleaned_csvs/report_for_client_{current_year_and_months}.csv")

In [40]:
report_for_client_by_month.to_csv(F"./output/cleaned_csvs/report_for_client_by_month_{current_year_and_months}.csv")

In [41]:
channels_no_spend.to_csv(F"./output/cleaned_csvs/channels_no_spend_{current_year_and_months}.csv")

## Exporting Reports to PDF Files

In [42]:
f = open('./output/reports/html/report_for_client.html','w')
a = report_for_client.to_html(col_space='100px')
f.write(a)
f.close()

pdfkit.from_file('./output/reports/html/report_for_client.html', './output/reports/pdfs/report_for_client.pdf')

True

In [43]:
f = open('./output/reports/html/report_for_client_by_month.html','w')
a = report_for_client_by_month.to_html(col_space='100px')
f.write(a)
f.close()

pdfkit.from_file('./output/reports/html/report_for_client_by_month.html', './output/reports/pdfs/report_for_client_by_month.pdf')

True

In [44]:
f = open('./output/reports/html/report_channels_no_spend.html','w')
a = channels_no_spend.to_html(col_space='100px')
f.write(a)
f.close()

pdfkit.from_file('./output/reports/html/report_channels_no_spend.html', './output/reports/pdfs/report_channels_no_spend.pdf')

True

# Viewing Reports

## Report by Network

In [45]:
report_for_client

,Purchases,Spend,Lift,Conversion Rate (Purchases/Lift)%,Cost Per Acquisition (Spend/Purchases),Cost Per Visitor (Spend/Lift)
Network,,,,,,
Bloomberg,2,4966.72,199,1.0,2483.36,24.96
Cnbc,15,6481.42,406,3.7,432.09,15.96
Cnbc World,0,1299.99,51,0.0,inf,25.49
Cnn,23,18114.35,1081,2.1,787.58,16.76
Comedy Central,2,7501.25,108,1.9,3750.62,69.46
Dateline,10,15288.11,862,1.2,1528.81,17.74
Dish Network,5,3033.33,1092,0.5,606.67,2.78
Fox Sports,2,4258.50,183,1.1,2129.25,23.27
Msnbc,16,11330.04,1123,1.4,708.13,10.09


## Report by Network and Month

In [46]:
report_for_client_by_month

Purchases     Spend  Lift  \
Network                  date                                    
Bloomberg                2017-09-30          1   4966.72   199   
                         2017-10-31          1      0.00     0   
Cnbc                     2017-09-30          5   6481.42   406   
                         2017-10-31         10      0.00     0   
Cnbc World               2017-09-30          0      0.00     0   
                         2017-10-31          0   1299.99    51   
Cnn                      2017-10-31         10   8954.75   574   
                         2017-09-30         13   9159.60   507   
Comedy Central           2017-09-30          0      0.00     0   
                         2017-10-31          2   7501.25   108   
Dateline                 2017-09-30          0   4211.62   252   
                         2017-10-31         10  11076.49   610   
Dish Network             2017-09-30          4   2513.09   976   
                         2017-10-31          1    520.24   116   
Fox Sports               2017-09-30          1   4258.50   183   
                         2017-10-31          1      0.00     0   
Msnbc                    2017-09-30         10   8091.15   701   
                         2017-10-31          6   3238.89   422   
Nfl Network              2017-09-30          2   5412.80   200   
                         2017-10-31          1      0.00     0   
One America News Network 2017-09-30          2   4966.80   648   
                         2017-10-31         14  19968.20  1724   
Science                  2017-09-30          1   7993.18   132   
                         2017-10-31          3      0.00     0   
Spectrum Sports          2017-09-30          2   5580.26   220   
                         2017-10-31          1      0.00     0   
Star Plus                2017-10-31          2   8338.98  1221   
                         2017-09-30          2   3441.58   562   
The History Channel      2017-10-31          1   1500.00    77   
                         2017-09-30          4   5778.30   253   
The Weather Channel      2017-10-31          0   7718.71   211   
                         2017-09-30          0      0.00     0   
Turner Network Tv        2017-09-30          2   5125.50   179   
                         2017-10-31          2      0.00     0   
Willow Tv                2017-10-31         29  27961.08  5076   
                         2017-09-30         27  19773.49  3138   
Zeetv                    2017-10-31          3  15429.12   819   
                         2017-09-30          2  10175.13   922   

                                     Conversion Rate (Purchases/Lift)%  \
Network                  date                                            
Bloomberg                2017-09-30                                0.5   
                         2017-10-31                                inf   
Cnbc                     2017-09-30                                1.2   
                         2017-10-31                                inf   
Cnbc World               2017-09-30                                0.0   
                         2017-10-31                                0.0   
Cnn                      2017-10-31                                1.7   
                         2017-09-30                                2.6   
Comedy Central           2017-09-30                                0.0   
                         2017-10-31                                1.9   
Dateline                 2017-09-30                                0.0   
                         2017-10-31                                1.6   
Dish Network             2017-09-30                                0.4   
                         2017-10-31                                0.9   
Fox Sports               2017-09-30                                0.5   
                         2017-10-31                                inf   
Msnbc                    2017-09-30                             

## Report of purchases for channels with no spend

In [48]:
channels_no_spend

,Purchases
Network,
Fox News,14.0
Other,13.0
Aapka Colors,6.0
Hgtv,6.0
Television Food Network,4.0
Sony Entertainment Tv,4.0
Espn,4.0
(Blank),3.0
Fyi,2.0


# Finish